In [1]:
import os
os.chdir("/dust3r")
print(os.getcwd())

/dust3r


In [12]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch
import json
import open3d as o3d

from dust3r.inference import inference_with_mask, create_gaussian_kernel
from dust3r.model import AsymmetricCroCo3DStereo
from dust3r.utils.image import load_images
from dust3r.image_pairs import make_pairs
from dust3r.cloud_opt import global_aligner, GlobalAlignerMode
from dust3r.cloud_opt.base_opt import global_alignment_loop
from masked_dust3r.scripts.utils.math import *
from masked_dust3r.scripts.utils.image import *


DATA_PATH = "/dust3r/masked_dust3r/data/jackal_irl_one_spin"
IMG_FILE_EXTENSION = ".png"
MASK_FILE_EXTENSION = ".png"
GAUSSIAN_SIGMA = 1.0
INIT_FRAMES = 30
NEW_FRAMES = 10
PREVIOUS_FRAMES = 40
TOTAL_FRAMES = 300

INIT_WEIGHT_FOCAL = 0
INIT_WEIGHT_Z = 0.01
INIT_WEIGHT_ROT = 0.01 
INIT_WEIGHT_TRANS_SMOOTHNESS = 0.00001 * 0
INIT_WEIGHT_ROT_SMOOTHNESS = 0.00001 * 0

NEW_WEIGHT_FOCAL = 0.1
NEW_WEIGHT_Z = 0.1
NEW_WEIGHT_ROT = 0.1
NEW_WEIGHT_TRANS_SMOOTHNESS = 0.00001
NEW_WEIGHT_ROT_SMOOTHNESS = 0.00001

USE_AVG_FOCAL = False

device = 'cuda'
batch_size = 1
schedule = 'cosine'
lr = 0.01
niter = 300

In [3]:
GAUSSIAN_SIGMA = 21.0
SIZE = int(GAUSSIAN_SIGMA * 3)

kernel = create_gaussian_kernel(SIZE, GAUSSIAN_SIGMA).to(device)

SIZE = 1
kernel = torch.ones(SIZE, SIZE).to(device)

/usr/local/lib/python3.10/dist-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [4]:
# Load the model

model_name = "checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth"
# you can put the path to a local checkpoint in model_name if needed
model = AsymmetricCroCo3DStereo.from_pretrained(model_name).to(device)

... loading model from checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth


instantiating : AsymmetricCroCo3DStereo(enc_depth=24, dec_depth=12, enc_embed_dim=1024, dec_embed_dim=768, enc_num_heads=16, dec_num_heads=12, pos_embed='RoPE100', patch_embed_cls='PatchEmbedDust3R', img_size=(512, 512), head_type='dpt', output_mode='pts3d', depth_mode=('exp', -inf, inf), conf_mode=('exp', 1, inf), landscape_only=False)
<All keys matched successfully>


In [5]:
images_array = []
masks_array = []

for i in range(10):
    images_array.append(os.path.join(DATA_PATH,"masked_images/{}{}".format(i,IMG_FILE_EXTENSION)))
    masks_array.append(os.path.join(DATA_PATH,"masks/{}{}".format(i,MASK_FILE_EXTENSION)))
images = load_images(images_array, size=512, verbose=True)
_,_,H,W = images[0]["img"].shape
masks = load_masks(masks_array, H, W, device)

>> Loading a list of 10 images
 - adding /dust3r/masked_dust3r/data/jackal_irl_one_spin/masked_images/0.png with resolution 640x480 --> 512x384
 - adding /dust3r/masked_dust3r/data/jackal_irl_one_spin/masked_images/1.png with resolution 640x480 --> 512x384
 - adding /dust3r/masked_dust3r/data/jackal_irl_one_spin/masked_images/2.png with resolution 640x480 --> 512x384
 - adding /dust3r/masked_dust3r/data/jackal_irl_one_spin/masked_images/3.png with resolution 640x480 --> 512x384
 - adding /dust3r/masked_dust3r/data/jackal_irl_one_spin/masked_images/4.png with resolution 640x480 --> 512x384
 - adding /dust3r/masked_dust3r/data/jackal_irl_one_spin/masked_images/5.png with resolution 640x480 --> 512x384
 - adding /dust3r/masked_dust3r/data/jackal_irl_one_spin/masked_images/6.png with resolution 640x480 --> 512x384
 - adding /dust3r/masked_dust3r/data/jackal_irl_one_spin/masked_images/7.png with resolution 640x480 --> 512x384
 - adding /dust3r/masked_dust3r/data/jackal_irl_one_spin/masked_i

In [6]:
pairs = make_pairs(images, scene_graph='swin-4', prefilter=None, symmetrize=True)
output = inference_with_mask(pairs, model, device, masks, kernel, batch_size=batch_size)

del pairs

>> Inference with model on 80 image pairs


100%|██████████| 80/80 [00:33<00:00,  2.40it/s]


In [7]:
#init_scene = global_aligner(output, device=device, mode=GlobalAlignerMode.ModularPointCloudOptimizer)
#loss = init_scene.compute_global_alignment(init="mst", niter=niter, schedule='cosine', lr=lr)

#scene = init_scene

scene = global_aligner(output, device=device, mode=GlobalAlignerMode.PlanePointCloudOptimizer, 
                        weight_focal = 0,
                        weight_z = INIT_WEIGHT_Z  * 0.1,
                        weight_rot = INIT_WEIGHT_ROT * 0.1,
                        weight_trans_smoothness = 0,
                        weight_rot_smoothness = 0)
#scene.im_poses = calculate_new_params(init_scene.im_poses,device)
#scene.im_focals = init_scene.im_focals
#print(scene.im_poses[0])
loss = scene.compute_global_alignment(init="mst", niter=niter, schedule=schedule, lr=lr)
#print(scene.im_poses[0])

#scene.weight_focal = INIT_WEIGHT_FOCAL
#scene.weight_z = INIT_WEIGHT_Z 
#scene.weight_rot = INIT_WEIGHT_ROT
#scene.weight_trans_smoothness = INIT_WEIGHT_TRANS_SMOOTHNESS 
#scene.weight_rot_smoothness = INIT_WEIGHT_ROT_SMOOTHNESS
#loss = global_alignment_loop(scene, niter=niter, schedule=schedule, lr=lr)


 init edge (5*,4*) score=1.2286036014556885
 init edge (5,6*) score=1.1971986293792725
 init edge (6,7*) score=1.1772550344467163
 init edge (7,9*) score=1.1710926294326782
 init edge (4,3*) score=1.1559910774230957
 init edge (5,2*) score=1.1506507396697998
 init edge (5,1*) score=1.1498658657073975
 init edge (4,0*) score=1.1146104335784912
 init edge (7,8*) score=1.1951526403427124
 init loss = 0.0008146763429977
Global alignement - optimizing for:
['pw_poses', 'im_depthmaps.0', 'im_depthmaps.1', 'im_depthmaps.2', 'im_depthmaps.3', 'im_depthmaps.4', 'im_depthmaps.5', 'im_depthmaps.6', 'im_depthmaps.7', 'im_depthmaps.8', 'im_depthmaps.9', 'im_poses.0', 'im_poses.1', 'im_poses.2', 'im_poses.3', 'im_poses.4', 'im_poses.5', 'im_poses.6', 'im_poses.7', 'im_poses.8', 'im_poses.9', 'im_focals.0', 'im_focals.1', 'im_focals.2', 'im_focals.3', 'im_focals.4', 'im_focals.5', 'im_focals.6', 'im_focals.7', 'im_focals.8', 'im_focals.9']


100%|██████████| 300/300 [01:39<00:00,  3.02it/s, lr=1.27413e-06 loss=6.06773e-05]


In [8]:
imgs = scene.imgs
focals = scene.get_focals()
poses = scene.get_im_poses()
pts3d = scene.get_pts3d()
confidence_masks = scene.get_masks()

tensor([[529.0613],
        [572.6783],
        [571.4208],
        [561.4349],
        [534.0599],
        [589.5187],
        [583.3734],
        [585.2063],
        [547.8608],
        [591.5915]], device='cuda:0', grad_fn=<ExpBackward0>)


In [9]:
#Check if pointclouds folder exists
#If exists, delete all files in the folder
if os.path.exists("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH)):
    for file in os.listdir("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH)):
        os.remove("{DATA_PATH}/pointclouds/{file}".format(DATA_PATH=DATA_PATH, file=file))
        
if not os.path.exists("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH)):
    os.makedirs("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH))

for i in range(len(images)):
    pointcloud = pts3d[i].detach().cpu().numpy()
    pointcloud = pointcloud.reshape(-1, 3)
    color = imgs[i].reshape(-1, 3)
    confidence_mask = confidence_masks[i].detach().cpu().numpy()
    confidence_mask = confidence_mask.reshape(-1)
    
    masked_pointcloud = []
    masked_color = []

    for j in range(len(confidence_mask)):
        if confidence_mask[j]:
            masked_pointcloud.append(pointcloud[j])
            masked_color.append(color[j])

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(masked_pointcloud)
    pcd.colors = o3d.utility.Vector3dVector(masked_color)
    o3d.io.write_point_cloud("{DATA_PATH}/pointclouds/pointcloud{i}.ply".format(DATA_PATH=DATA_PATH, i=i), pcd)

In [39]:
OPENGL = np.array([[1, 0, 0, 0],
                    [0, -1, 0, 0],
                    [0, 0, -1, 0],
                    [0, 0, 0, 1]])

img = Image.open(images_array[0])
width, height = img.size

RESCALE_FACTOR = 512/width

#Create transform file
transforms = {}
transforms["camera_model"] = "OPENCV"

if USE_AVG_FOCAL:
    averge_focal = focals.sum()/len(focals)
    transforms["fl_x"] = averge_focal.item()
    transforms["fl_y"] = averge_focal.item()

    #Find size of images
    img = Image.open(images_array[0])
    width, height = img.size
    transforms["w"] = width * RESCALE_FACTOR
    transforms["h"] = height * RESCALE_FACTOR
    transforms["cx"] = width/2 * RESCALE_FACTOR
    transforms["cy"] = height/2 * RESCALE_FACTOR

transforms["frames"] = []

for i in range(len(poses)):
    if not((confidence_masks[i]==0).all()):
        frame = {}
        frame["file_path"] = "/".join(images_array[i].split("/")[-2:])
        frame["transform_matrix"] = poses[i].detach().cpu().numpy()
        frame["transform_matrix"] = np.linalg.inv(frame["transform_matrix"])
        frame["transform_matrix"] = np.matmul(OPENGL, frame["transform_matrix"])
        #print(frame["transform_matrix"])
        #frame["transform_matrix"] = np.linalg.inv(frame["transform_matrix"])
        #print(frame["transform_matrix"])
        frame["transform_matrix"] = frame["transform_matrix"].tolist()
        #print(frame["transform_matrix"])
        frame["mask_path"] = "/".join(masks_array[i].split("/")[-2:])
        
        if not USE_AVG_FOCAL:
            frame["fl_x"] = focals[i].item()
            frame["fl_y"] = focals[i].item()
            img = Image.open(images_array[i])
            width, height = img.size
            frame["w"] = width * RESCALE_FACTOR
            frame["h"] = height * RESCALE_FACTOR
            frame["cx"] = width/2 * RESCALE_FACTOR
            frame["cy"] = height/2 * RESCALE_FACTOR
        
        transforms["frames"].append(frame)
        

#Save transform file
with open("{}/transforms.json".format(DATA_PATH), 'w') as f:
    json.dump(transforms, f, indent=4)